In [23]:
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor



In [24]:
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


train = pd.read_csv('train_v9rqX0R.csv')
X_train = train.drop(['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'], axis=1)
y_train = train['Item_Outlet_Sales']

imp_mode =  SimpleImputer(strategy='most_frequent')
imp_mean =  SimpleImputer(strategy='median')
ohc = OneHotEncoder(sparse_output=False)


In [25]:
ct_imp = make_column_transformer((imp_mode, make_column_selector(dtype_include=object)),
                             (imp_mean, make_column_selector(dtype_include=['int64','float64'])), verbose_feature_names_out=False).set_output(transform='pandas')

ct_enc = make_column_transformer((ohc, make_column_selector(dtype_include=['object'])),
                                  ('passthrough', make_column_selector(dtype_include=['int64','float64'])),verbose_feature_names_out=False).set_output(transform='pandas')

dtr = DecisionTreeRegressor(random_state=23)
pipe = Pipeline([('IMPUTE',ct_imp),('ENC',ct_enc),('Tree',dtr)])

pipe.fit(X_train,y_train)

Pipeline(steps=[('IMPUTE',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f668031bb90>),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f668031b710>)],
                                   verbose_feature_...
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f6680164dd0>),
                                                 ('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f6680164750>)],
                                   verbose_feature_names_out=False)),
                ('Tree', DecisionTreeRegressor(random_state=23))])

In [26]:
test = pd.read_csv('test_AbJTz2l.csv')
X_test = test.drop(['Item_Identifier','Outlet_Identifier'], axis=1)
predictions = pipe.predict(X_test)

In [27]:


submit = pd.DataFrame({'Item_Identifier':test['Item_Identifier'],'Outlet_Identifier':test['Outlet_Identifier'],'Item_Outlet_Sales':predictions})
submit.to_csv('submission_for_Big_Mart.csv',index=False)

In [35]:
from sklearn.ensemble import BaggingRegressor


dtr = DecisionTreeRegressor(random_state=23)
bagg = BaggingRegressor(dtr, n_estimators=15,random_state=23)
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
pipe = Pipeline([('IMPUTE',ct_imp),('ENC',ct_enc),('Bag',bagg)])
print(bagg.get_params())
params = {'Bag__estimator__max_depth':[None, 3, 5],'Bag__estimator__min_samples_split':[2,5,10],'Bag__estimator__min_samples_leaf':[1,5,10],'Bag__n_estimators':[10,30, 50]}
gcv = GridSearchCV(pipe, param_grid=params,cv=kfold,verbose=3, scoring='r2')
gcv.fit(X_train,y_train)
print(gcv.best_params_)
print(gcv.best_score_)

{'base_estimator': 'deprecated', 'bootstrap': True, 'bootstrap_features': False, 'estimator__ccp_alpha': 0.0, 'estimator__criterion': 'squared_error', 'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__random_state': 23, 'estimator__splitter': 'best', 'estimator': DecisionTreeRegressor(random_state=23), 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 15, 'n_jobs': None, 'oob_score': False, 'random_state': 23, 'verbose': 0, 'warm_start': False}
Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV 1/5] END Bag__estimator__max_depth=None, Bag__estimator__min_samples_leaf=1, Bag__estimator__min_samples_split=2, Bag__n_estimators=10;, score=0.519 total time=   0.8s
[CV 2/5] END Bag__estimator__max_depth=None, Bag__estimator__min_samples_leaf=1, Bag__estimator_